# Семантическая сегментация изображений на основе полностью сверточной нейронной сети

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Параметры</a></li>
        <li><a href="#2">Загрузка датасета KittyRoad</a></li>
        <li><a href="#3">Загрузка сверточной нейронной сети классификации VGG16</a></li>
        <li><a href="#4">Работа с данными</a></li>
        <li><a href="#5">Полностью сверточная нейронная сеть FCN8</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#5a">Кодер</a></li>
                <li><a href="#5b">Декодер</a></li>
                <li><a href="#5c">Обучение и тестирование</a></li>
            </ol>
        </li>
        <li><a href="#6">Источники</a>
    </ol>
</div>

Импорт библиотек

In [ ]:
import random
import time
import tensorflow as tf
import os
from tqdm import tqdm
from urllib.request import urlretrieve
import shutil
import zipfile
from glob import glob
from glob import glob1
import re
import numpy as np
import scipy.misc
import matplotlib.pyplot as plt

#### !!! Сеть будет обучаться достаточно долго. Данную тетрадь можно запустить в облачном сервисе Google Colaboratory https://colab.research.google.com/ и значительно сократить время обучения.

<a name="1"></a>
## 1. Параметры

In [1]:
# Путь к предобученной модели VGG16
vgg16_path = "./lib/vgg16"
# Путь к датасету
dataset_path = "./data/fcn8-kittiroad/data_road"
# Путь для сохранения сегментированных картинок
out_path = "./out/fcn8-kittiroad"
# Путь для сохранения обученной нейронной сети
model_path="./models/fcn8-kittiroad/fcn8"
# Количество классов сегментации (дорога и не дорога)
num_classes = 2
# Размер входного изображения
image_shape = (160, 576)
# Размер батча
batch_size = 16
# Количество эпох обучения
epochs = 15
# Скорость(шаг) обучения
lr = 0.0001
# Уровень dropout-регуляризации
dropout = 0.8

<a name="2"></a>
## 2. Загрузка датасета KittiRoad

In [ ]:
! wget -P data/fcn8-kittiroad/ https://s3.eu-central-1.amazonaws.com/avg-kitti/data_road.zip
! unzip data/fcn8-kittiroad/data_road.zip -d data/fcn8-kittiroad/

<a name="3"></a>
## 3. Загрузка сверточной нейронной сети классификации VGG16

Код для загрузки предобученной сверточной нейронной сети классификации VGG16

In [ ]:
class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num


def download_pretrained_vgg(data_dir):
    vgg_filename = 'vgg.zip'
    vgg_path = os.path.join(data_dir, 'vgg')
    vgg_files = [
        os.path.join(vgg_path, 'variables/variables.data-00000-of-00001'),
        os.path.join(vgg_path, 'variables/variables.index'),
        os.path.join(vgg_path, 'saved_model.pb')]

    missing_vgg_files = [vgg_file for vgg_file in vgg_files if not os.path.exists(vgg_file)]
    if missing_vgg_files:
        if os.path.exists(vgg_path):
            shutil.rmtree(vgg_path)
        os.makedirs(vgg_path)

        print('Downloading pre-trained vgg model...')
        with DLProgress(unit='B', unit_scale=True, miniters=1) as pbar:
            urlretrieve(
                'https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/vgg.zip',
                os.path.join(vgg_path, vgg_filename),
                pbar.hook)

        print('Extracting model...')
        zip_ref = zipfile.ZipFile(os.path.join(vgg_path, vgg_filename), 'r')
        zip_ref.extractall(data_dir)
        zip_ref.close()

        os.remove(os.path.join(vgg_path, vgg_filename))

Загрузка VGG16

In [ ]:
download_pretrained_vgg(vgg16_path)

<a name="4"></a>
## 4. Работа с данными

In [ ]:
# функция для считывания пары изображений датасета-исходное изображение и размеченное
def get_dataset_image(dataset_folder, image_shape):
    data_folder = os.path.join(dataset_folder, "training")
    
    image_paths = glob(os.path.join(data_folder, "image_2", "*.png"))
    label_paths = {
        re.sub(r"_(lane|road)_", "_", os.path.basename(path)): path
        for path in glob(os.path.join(data_folder, "gt_image_2", "*_road_*.png"))}
    background_color = np.array([255, 0, 0])
    
    image_file = np.random.choice(image_paths)
    
    gt_image_file = label_paths[os.path.basename(image_file)]
                
    image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)
    gt_image = scipy.misc.imresize(scipy.misc.imread(gt_image_file), image_shape)
    
    return np.array(image), np.array(gt_image)


# вывод рандомного обучающего примера из датасета
def show_sample(path, image_shape):
    x_train_img, y_train_img = get_dataset_image(path, image_shape)
    fig=plt.figure(figsize=(25, 25))
    fig.add_subplot(1,2,1)
    plt.imshow(x_train_img)
    fig.add_subplot(1,2,2)
    plt.imshow(y_train_img)
    plt.show()

Вывод пары (исходное изображение, сегментированное изображение) из обучающего датасета

In [ ]:
show_sample(dataset_path, (375, 1242))

Для обучения сети нужно произвести преобразование каждого пикселя размеченного изображения из обучающего набора в one-hot encoding

Исходное представление
<img src="images/fcn8-kittiroad/seg.png" width="900px">
<br>
One-hot encoding представление
<img src="images/fcn8-kittiroad/one_hot_seg.png" width="900px">

In [ ]:
# генератор изображений для обучения сети
def gen_batch_function(dataset_folder, image_shape):
    data_folder = os.path.join(dataset_folder, "training")
    # получение очередного батча
    def get_batches_fn(batch_size):
        image_paths = glob(os.path.join(data_folder, 'image_2', '*.png'))
        label_paths = {
            re.sub(r'_(lane|road)_', '_', os.path.basename(path)): path
            for path in glob(os.path.join(data_folder, 'gt_image_2', '*_road_*.png'))}
        background_color = np.array([255, 0, 0])

        random.shuffle(image_paths)
        # расчет количества батчей исходя из размера батча
        for batch_i in range(0, len(image_paths), batch_size):
            images = []
            gt_images = []
            # загрузка изображений в батч
            for image_file in image_paths[batch_i:batch_i+batch_size]:
                gt_image_file = label_paths[os.path.basename(image_file)]

                image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)
                gt_image = scipy.misc.imresize(scipy.misc.imread(gt_image_file), image_shape)

                # Преобразуем из RGB размерности в one-hot encoding формат
                # т.е. каждый пиксель будет категориальным
                # [[[0,1]-категориальный пиксель в строке,[1,0],[0,1]...]-строка, [[0,1],[1,0],[0,1]...], ...]
                gt_bg = np.all(gt_image == background_color, axis=2)
                gt_bg = gt_bg.reshape(*gt_bg.shape, 1)
                gt_image = np.concatenate((gt_bg, np.invert(gt_bg)), axis=2)

                images.append(image)
                gt_images.append(gt_image)

            yield np.array(images), np.array(gt_images)
    return get_batches_fn


# генератор изображений для теста сети
def gen_test_output(sess, logits, keep_prob, image_pl, data_folder, image_shape):
    for image_file in glob(os.path.join(data_folder, 'image_2', '*.png')):
        image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)
        #image = skimage.img_as_ubyte(transform.resize(scipy.misc.imread(image_file), image_shape))

        # прогоняем изображение через модель
        im_softmax = sess.run(
            [tf.nn.softmax(logits)],
            {keep_prob: 1.0, image_pl: [image]})
        im_softmax = im_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
        # применяем порог к вероятностям softmax
        segmentation = (im_softmax > 0.5).reshape(image_shape[0], image_shape[1], 1)
        # формируем маску сегментации
        mask = np.dot(segmentation, np.array([[0, 255, 0, 127]]))
        mask = scipy.misc.toimage(mask, mode="RGBA")
        # накладываем маску сегментации на изображение
        street_im = scipy.misc.toimage(image)
        street_im.paste(mask, box=None, mask=mask)

        yield os.path.basename(image_file), np.array(street_im)

        
# сегментирование тестовых изображений с помощью обученной сети и сохранение
def save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image):
    output_dir = os.path.join(runs_dir, str(time.time()))
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)

    print('Saving test images to: {}, please wait...'.format(output_dir))
    image_outputs = gen_test_output(
        sess, logits, keep_prob, input_image, os.path.join(data_dir, 'testing'), image_shape)
    for name, image in image_outputs:
         scipy.misc.imsave(os.path.join(output_dir, name), image)

    print('All images are saved!')

# сегментация рандомного образца тестовой выборки
def test_random_sample(data_dir, sess, image_shape, logits, keep_prob, image_pl):
    images = glob(os.path.join(os.path.join(data_dir, 'testing'), 'image_2', '*.png'))
    image_file = np.random.choice(images)
    image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)
    
    # прогоняем изображение через модель
    im_softmax = sess.run(
        [tf.nn.softmax(logits)],
        {keep_prob: 1.0, image_pl: [image]})
    im_softmax = im_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
    
    # применяем порог к вероятностям softmax
    segmentation = (im_softmax > 0.5).reshape(image_shape[0], image_shape[1], 1)
    
    # формируем маску сегментации
    mask = np.dot(segmentation, np.array([[0, 255, 0, 127]]))
    mask = scipy.misc.toimage(mask, mode="RGBA")
    
    # накладываем маску сегментации на изображение
    street_im = scipy.misc.toimage(image)
    street_im.paste(mask, box=None, mask=mask)
    
    return os.path.basename(image_file), np.array(street_im)

<a name="5"></a>
## 5. Полностью сверточная нейронная сеть FCN8

Схема полностью сверточной нейронной сети FCN8

<img src="images/fcn8-kittiroad/fcn.png" width="700px">

<img src="images/fcn8-kittiroad/fcn_8.png" width="900px">

FCN8 имеет архитектуру кодер-декодер (encoder-decoder)<br>
Кодер - предварительно обученная нейронная сеть классификации VGG16 без полносязных слоев<br>
Декодер - транспонированная свертка (transpose convolution) и соединения (skip-connections) с соответствующими картами кодера

<a name="5a"></a>
### a. Кодер 

Структура сети VGG16 (для кодера FC слои исключаются из сети)

<img src="images/fcn8-kittiroad/vgg16.png" width="700px">

<a name="5b"></a>
### b. Декодер

Организация работы декодера

<img src="images/fcn8-kittiroad/fcn8.png" width="1000px">

Получим входной слой, регуляризацию, выходные слои 3,4,7 сети VGG16 (в загруженной ранее сети vgg16 уже убраны FC слои и вместо них добавлены слои свертки 6-7 в соответствии со схемой сети FCN8)

In [ ]:
def load_vgg(sess, vgg_path):
    tf.saved_model.loader.load(sess, ['vgg16'], vgg_path)
    image_input = tf.get_default_graph().get_tensor_by_name('image_input:0')
    keep_prob = tf.get_default_graph().get_tensor_by_name('keep_prob:0')
    vgg_layer3_out = tf.get_default_graph().get_tensor_by_name('layer3_out:0')
    vgg_layer4_out = tf.get_default_graph().get_tensor_by_name('layer4_out:0')
    vgg_layer7_out = tf.get_default_graph().get_tensor_by_name('layer7_out:0')

    return image_input, keep_prob, vgg_layer3_out, vgg_layer4_out, vgg_layer7_out

Формируем декодер

In [ ]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    # свертка 1х1 по всем картам признаков слоев 3,4,7 сети VGG16
    # применяется для лучшего обучения сети, а также повышает точность сегментации
    vgg_layer7_logits = tf.layers.conv2d(
        vgg_layer7_out, num_classes, kernel_size=1,
        kernel_initializer= tf.random_normal_initializer(stddev=0.01),
        kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-4), name='vgg_layer7_logits')
    vgg_layer4_logits = tf.layers.conv2d(
        vgg_layer4_out, num_classes, kernel_size=1,
        kernel_initializer= tf.random_normal_initializer(stddev=0.01),
        kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-4), name='vgg_layer4_logits')
    vgg_layer3_logits = tf.layers.conv2d(
        vgg_layer3_out, num_classes, kernel_size=1,
        kernel_initializer= tf.random_normal_initializer(stddev=0.01),
        kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-4), name='vgg_layer3_logits')

    # transpose convolutions и skip-connections
    fcn_decoder_layer1 = tf.layers.conv2d_transpose(
        vgg_layer7_logits, num_classes, kernel_size=4, strides=(2, 2),
        padding='same',
        kernel_initializer= tf.random_normal_initializer(stddev=0.01),
        kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-4), name='fcn_decoder_layer1')

    fcn_decoder_layer2 = tf.add(
        fcn_decoder_layer1, vgg_layer4_logits, name='fcn_decoder_layer2')

    fcn_decoder_layer3 = tf.layers.conv2d_transpose(
        fcn_decoder_layer2, num_classes, kernel_size=4, strides=(2, 2),
        padding='same',
        kernel_initializer= tf.random_normal_initializer(stddev=0.01),
        kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-4), name='fcn_decoder_layer3')

    fcn_decoder_layer4 = tf.add(
        fcn_decoder_layer3, vgg_layer3_logits, name='fcn_decoder_layer4')
    fcn_decoder_output = tf.layers.conv2d_transpose(
        fcn_decoder_layer4, num_classes, kernel_size=16, strides=(8, 8),
        padding='same',
        kernel_initializer= tf.random_normal_initializer(stddev=0.01),
        kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-4), name='fcn_decoder_layer4')

    return fcn_decoder_output

<a name="5c"></a>
### с. Обучение и тестирование

Функция оптимизации - в качестве функции потерь взята попиксельная кросс-энтропийная потеря, оптимизатор Адам

<img src="images/fcn8-kittiroad/pix_ce_loss.png" width="1000px">

In [ ]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    # преобразование в 2D one-hot encoding представление
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    correct_label = tf.reshape(correct_label, (-1, num_classes))
   
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=correct_label))
    optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
    train_op = optimizer.minimize(cross_entropy_loss)

    return logits, train_op, cross_entropy_loss

Функция обучения нейронной сети FCN8

In [ ]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    sess.run(tf.global_variables_initializer())

    print("Training...")
    print()
    for i in range(epochs):
        print("Epoch {} ...".format(i+1))
        training_loss = 0
        training_samples = 0
        starttime = time.clock()
        for image, label in get_batches_fn(batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss],
                               feed_dict={input_image: image, correct_label: label,
                                          keep_prob: dropout, learning_rate: lr})
            training_samples += 1
            training_loss += loss

        training_loss /= training_samples
        endtime = time.clock()
        training_time = endtime-starttime

        print("Loss: = {:.3f}\n".format(training_loss))

Запускаем пайплайн создания сети FCN8 и ее обучения

In [ ]:
# получение сессии
sess = tf.Session()
# генератор изображений для обучения
get_batches_fn = gen_batch_function(dataset_path, image_shape)

# TF placeholders
correct_label = tf.placeholder(tf.int32, [None, None, None, num_classes], name='correct_label')
learning_rate = tf.placeholder(tf.float32, name='learning_rate')

# Получение входного слоя, регуляризации, выходных слоев 3,4,7 сети VGG16
input_image, keep_prob, vgg_layer3_out, vgg_layer4_out, vgg_layer7_out = load_vgg(sess, vgg16_path + "/vgg")

# Построение FCN8 используя слои VGG16
nn_last_layer = layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes)

logits, train_op, cross_entropy_loss = optimize(nn_last_layer, correct_label, learning_rate, num_classes)

In [ ]:
# Обучение сети
train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image, 
         correct_label, keep_prob, learning_rate)

In [ ]:
# Сохранение сети в файл
#saver = tf.train.Saver()
#save_path = saver.save(sess, model_path)
#print("Model is saved to file: %s" % save_path)

Вывод рандомного тестового образца сегментированного обученной нейронной сетью FCN8

In [ ]:
test_name, test_img = test_random_sample(dataset_path, sess, image_shape, logits, keep_prob, input_image)
plt.figure(figsize=(20, 20))
plt.imshow(test_img)

# сегментация всех тестовых изображений с помощью обученной fcn8 и их сохранение
# save_inference_samples(out_path, dataset_path, sess, image_shape, logits, keep_prob, input_image)

In [ ]:
sess.close()

<a name="6"></a>
## 6. Источники

Датасет KittiRoad: https://s3.eu-central-1.amazonaws.com/avg-kitti/data_road.zip<br>
Предобученная сверточная нейронная сеть классификации VGG16: https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/vgg.zip<br>
Road Segmentation: https://junshengfu.github.io/semantic_segmentation/<br>
An Overview of semantic image segmentation: https://www.jeremyjordan.me/semantic-segmentation/<br>
Fully Convolutional Networks for Semantic Segmentation: https://people.eecs.berkeley.edu/~jonlong/long_shelhamer_fcn.pdf <br>